In [1]:
import os
import glob

import fiftyone as fo

from benthic_mapping.download_media import MediaDownloader
from benthic_mapping.fiftyone_clustering import FiftyOneDatasetViewer
from benthic_mapping.download_labeled_data import LabeledDataDownloader

# Download Media (frames) from Tator

In [2]:
# Initialize the downloader with the required parameters
downloader = MediaDownloader(
    api_token=os.getenv("TATOR_TOKEN"),
    project_id=155,
    output_dir="../Data"
)

# Download the media
media_ids = ["14759824"]
downloader.download_data(media_ids, 
                         convert=False, 
                         extract=True, 
                         every_n_seconds=3)

NOTE: Authentication successful for jordan.pierce
NOTE: Media GL2301_VID_20230725T145731Z_D015_DROPCAM_HIGH.mp4 downloaded successfully to ../Data\Raw_Videos\GL2301_VID_20230725T145731Z_D015_DROPCAM_HIGH_converted.mp4
NOTE: Extracting frames from GL2301_VID_20230725T145731Z_D015_DROPCAM_HIGH.mp4...
Video duration: 381.88 seconds
Extracting 127 frames using 16 workers...
Successfully extracted 127 frames to ../Data\Extracted_Frames\GL2301_VID_20230725T145731Z_D015_DROPCAM_HIGH_converted


['../Data\\Raw_Videos\\GL2301_VID_20230725T145731Z_D015_DROPCAM_HIGH_converted.mp4']

In [3]:
downloader.media_path_map

{'14759824': {'original': '../Data\\Raw_Videos\\GL2301_VID_20230725T145731Z_D015_DROPCAM_HIGH_converted.mp4',
  'converted': None,
  'frames': '../Data\\Extracted_Frames\\GL2301_VID_20230725T145731Z_D015_DROPCAM_HIGH_converted'}}

# View Clustered Frames from Tator using Fiftyone

In [4]:
# Initialize the viewer with the path to the directory containing images
viewer = FiftyOneDatasetViewer(image_dir=downloader.media_path_map['14759824']['frames'])

# Process the dataset to create the FiftyOne dataset and generate the UMAP visualization
viewer.process_dataset()

Overwriting existing dataset: GL2301_VID_20230725T145731Z_D015_DROPCAM_HIGH_converted


Processing images: 100%|██████████| 127/127 [00:06<00:00, 20.27it/s]

   1% ||----------------|   1/127 [18.3ms elapsed, 2.3s remaining, 54.8 samples/s] 

 100% |█████████████████| 127/127 [99.4ms elapsed, 0s remaining, 1.3K samples/s]   
Computing embeddings...
Computing UMAP visualization...
Generating visualization...


c:\Users\jordan.pierce\.conda\envs\bm\lib\site-packages\sklearn\utils\deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
c:\Users\jordan.pierce\.conda\envs\bm\lib\site-packages\umap\umap_.py:1952: UserWarning: n_jobs value 1 overridden to 1 by setting random_state. Use no seed for parallelism.
  warn(


UMAP(n_jobs=1, random_state=51, verbose=True)
Fri Feb 28 23:04:42 2025 Construct fuzzy simplicial set
Fri Feb 28 23:04:43 2025 Finding Nearest Neighbors
Fri Feb 28 23:04:50 2025 Finished Nearest Neighbor Search
Fri Feb 28 23:04:54 2025 Construct embedding


Epochs completed:   0%|            0/500 [00:00]

	completed  0  /  500 epochs
	completed  50  /  500 epochs
	completed  100  /  500 epochs
	completed  150  /  500 epochs
	completed  200  /  500 epochs
	completed  250  /  500 epochs
	completed  300  /  500 epochs
	completed  350  /  500 epochs
	completed  400  /  500 epochs
	completed  450  /  500 epochs
Fri Feb 28 23:04:55 2025 Finished embedding


In [ ]:
# Launch the FiftyOne app
try:
    session = fo.launch_app(viewer.dataset)
except:
    session = fo.launch_app(viewer.dataset)

# Download Labeled Data from Tator

In [2]:
# Set parameters
api_token = os.getenv("TATOR_TOKEN")
project_id = 155  # 70

# Search string comes from Tator's Data Metadata Export utility
search_string = "eyJtZXRob2QiOiJBTkQiLCJvcGVyYXRpb25zIjpbeyJhdHRyaWJ1dGUiOiIkY3JlYXRlZF9ieSIsIm9wZXJhdGlvbiI6ImVxIiwiaW52ZXJzZSI6ZmFsc2UsInZhbHVlIjo1MDZ9LHsibWV0aG9kIjoiT1IiLCJvcGVyYXRpb25zIjpbeyJhdHRyaWJ1dGUiOiIkdHlwZSIsIm9wZXJhdGlvbiI6ImVxIiwiaW52ZXJzZSI6ZmFsc2UsInZhbHVlIjo0NjB9LHsiYXR0cmlidXRlIjoiJHR5cGUiLCJvcGVyYXRpb24iOiJlcSIsImludmVyc2UiOmZhbHNlLCJ2YWx1ZSI6NTUzfV19XX0="

# Demo for downloading labeled data
frac = 0.1

dataset_name = "benthic_demo"
output_dir = "../Data/Labeled_Data"

label_field = "Label"
task = "detect"  # "detect" for bboxes or "segment" for polygons (if applicable)

In [3]:
# Create a downloader for the labeled data
downloader = LabeledDataDownloader(api_token,
                                   project_id=project_id,
                                   search_string=search_string,
                                   frac=frac,
                                   output_dir=output_dir,
                                   dataset_name=dataset_name,
                                   label_field=label_field,
                                   task=task)

NOTE: Authentication successful for jordan.pierce
NOTE: Search string saved to e:\Benthic-Mapping\Data\Labeled_Data\benthic_demo\search_string.txt


In [4]:
# Download the labeled data
downloader.download_data()

NOTE: Querying Tator for labeled data
NOTE: Found 8645 localizations


Processing Query: 100%|██████████| 8645/8645 [00:00<00:00, 11541.39it/s]


NOTE: Found 904 localizations after sampling
NOTE: Data saved to e:\Benthic-Mapping\Data\Labeled_Data\benthic_demo\data.json
NOTE: Downloading images to e:\Benthic-Mapping\Data\Labeled_Data\benthic_demo\images


NOTE: Images downloaded to e:\Benthic-Mapping\Data\Labeled_Data\benthic_demo\images
